In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [3]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-27 15:00:31.386876: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 15:00:31.454980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [4]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [5]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [17]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [18]:
init(model)

In [19]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack

adapter_name= "standalone-mlm-target"
adapter_config = AdapterConfig.load("seq_bn", reduction_factor=2, leave_out=[])
model.add_adapter(adapter_name, config=adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 4137984 || all params: 70531386 || trainable%: 5.866868970928772


In [20]:
model.active_adapters

Stack[standalone-mlm-target]

In [21]:
model.adapter_summary(as_dict=True)

[{'name': 'standalone-mlm-target',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': True,
  '%param': 5.343131581992825},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [22]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


distilbert.transformer.layer.0.output_adapters.adapters.standalone-mlm-target.adapter_down.0.weight
distilbert.transformer.layer.0.output_adapters.adapters.standalone-mlm-target.adapter_down.0.bias
distilbert.transformer.layer.0.output_adapters.adapters.standalone-mlm-target.adapter_up.weight
distilbert.transformer.layer.0.output_adapters.adapters.standalone-mlm-target.adapter_up.bias
distilbert.transformer.layer.1.output_adapters.adapters.standalone-mlm-target.adapter_down.0.weight
distilbert.transformer.layer.1.output_adapters.adapters.standalone-mlm-target.adapter_down.0.bias
distilbert.transformer.layer.1.output_adapters.adapters.standalone-mlm-target.adapter_up.weight
distilbert.transformer.layer.1.output_adapters.adapters.standalone-mlm-target.adapter_up.bias
distilbert.transformer.layer.2.output_adapters.adapters.standalone-mlm-target.adapter_down.0.weight
distilbert.transformer.layer.2.output_adapters.adapters.standalone-mlm-target.adapter_down.0.bias
distilbert.transformer.lay

In [23]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(unsupervised_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 52350
})

In [24]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [25]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [26]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 18436
})

In [27]:
print(tokenizer.decode(lm_datasets[1]["input_ids"]))
print(tokenizer.decode(lm_datasets[1]['labels']))
print(len(lm_datasets[-1]['labels']))


1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the minoans used images of birds and fish to represent how they thought they would be reincarnated. [SEP] [CLS] shinjuku has tokyo's largest discount camera stores, although these rarely
1874 exhibition of impressionism. [SEP] pissarro never walked on the boulevard des capucines. [SEP] [CLS] a church was once built over this spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the old church to find the pool. [SEP] [CLS] room 13 contains a number of painted minoan sarcophagi, many decorated with the images of fish or birds. [SEP] the

In [28]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] foreigners from [MASK] entire ottoman empire flooded [MASK] gala [MASK] and pera, attracted by the wealth [MASK] sophi [MASK]ation of the capital. [SEP] no foreigners came from other lands to gala heard. [SEP] [CLS] muslim rule was largely [MASK], and continued peacefully for nearly four centuries with a joint christian - muslim protectorate of holy places. [SEP] christians and muslims [MASK] fierce [MASK], especially when it came to holy [MASK]. [SEP] [CLS] on boulevard [MASK] capucines ins [MASK]trace the footsteps of renoir, [MASK]t, [MASK] pissarro as they took their paintings to [MASK]ar'[MASK] house, signposted at number 35, for the historic'

'>>> 1874 exhibition [MASK] impressionism. [SEP] pissarro never walked on the boulevard des [MASK]ucines [MASK] [SEP] [CLS] a church was once [MASK] over [MASK] spot, but today the minaret of a small mosque is the best landmark for finding the pool. [SEP] just look for the [MASK] church [MASK] find the pool. [SEP] [CLS] room 13 

In [29]:
reload(fn)
samples = [lm_datasets[i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] foreigners from the entire ottoman empire flooded into galata and [MASK] [MASK], attracted by the wealth and sophistication of the capital. [SEP] no foreigners [MASK] from other [MASK] to galata [MASK] [SEP] [CLS] muslim rule was largely [MASK], and continued peacefully for nearly four centuries with a joint christian - [MASK] protectorate [MASK] holy places. [SEP] christians and muslims [MASK] [MASK] [MASK], especially when it came to holy [MASK]. [SEP] [CLS] on boulevard des capucines, retrace the [MASK] of [MASK] [MASK], manet, and pissarro as they took their paintings [MASK] nadar's house, signposted at number 35, for the historic'

'>>> 1874 exhibition of [MASK] [MASK]. [SEP] pissarro never [MASK] on the boulevard [MASK] [MASK] [MASK] [MASK]. [SEP] [CLS] a church was once built over this spot, but [MASK] the minaret of [MASK] small [MASK] is the best landmark for finding the pool. [SEP] just look for the [MASK] church [MASK] find the pool. [SEP] [CLS] [MASK] 13 contain

In [30]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 16592
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1844
    })
})

In [32]:

reload(fn)
trainer = fn.train_mlm_model(model,"mlm-after-coral",data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [33]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 22.55


In [34]:
trainer.train()

  0%|          | 0/5190 [00:00<?, ?it/s]

{'loss': 2.5055, 'learning_rate': 9.961620469083156e-05, 'epoch': 1.0}


  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 2.172438144683838, 'eval_runtime': 1.9584, 'eval_samples_per_second': 941.572, 'eval_steps_per_second': 29.616, 'epoch': 1.0}
{'loss': 2.2454, 'learning_rate': 8.857142857142857e-05, 'epoch': 2.0}


  0%|          | 0/58 [00:00<?, ?it/s]

...

{'eval_loss': 1.9098109006881714, 'eval_runtime': 1.9875, 'eval_samples_per_second': 927.777, 'eval_steps_per_second': 29.182, 'epoch': 9.0}
{'loss': 2.0117, 'learning_rate': 2.1321961620469084e-07, 'epoch': 9.98}


  0%|          | 0/58 [00:00<?, ?it/s]

Overwriting existing adapter 'standalone-mlm-target'.


{'eval_loss': 1.9160041809082031, 'eval_runtime': 1.9746, 'eval_samples_per_second': 933.871, 'eval_steps_per_second': 29.373, 'epoch': 10.0}
{'train_runtime': 341.0777, 'train_samples_per_second': 486.458, 'train_steps_per_second': 15.216, 'train_loss': 2.127591548765326, 'epoch': 10.0}


TrainOutput(global_step=5190, training_loss=2.127591548765326, metrics={'train_runtime': 341.0777, 'train_samples_per_second': 486.458, 'train_steps_per_second': 15.216, 'train_loss': 2.127591548765326, 'epoch': 10.0})

In [35]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/58 [00:00<?, ?it/s]

>>> Perplexity: 6.72


In [36]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [37]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [38]:
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my tour.
>>> i cancelled my plans.
>>> i cancelled my appointment.
>>> i cancelled my flight.
>>> i cancelled my visit.
